## Lab 2: Adding Memory to Multi-Agent Customer Support System

### Overview

In Lab 1, you built a multi-agent customer support system with an orchestrator and specialized agents. While this system works well for individual sessions, real-world customer support needs persistent memory to provide personalized experiences.

When we run **Multi-Agent Systems in Production**, we need:
- **Cross-Agent Memory Sharing**: Agents remember customer context across handoffs
- **Persistent Customer Profiles**: Long-term learning about customer preferences
- **Intelligent Routing Memory**: Orchestrator learns optimal routing patterns
- **Technical Solution History**: Knowledge base agent remembers successful solutions

**Workshop Progress:**
- **Lab 1 (Done)**: Multi-Agent Foundation - Built orchestrator with specialized agents
- **Lab 2 (Current)**: Multi-Agent Memory - Add persistent memory across agents
- **Lab 3**: Multi-Agent Gateway - Secure tool sharing and identity management
- **Lab 4**: Multi-Agent Runtime - Deploy with observability and monitoring
- **Lab 5**: Multi-Agent Frontend - Build customer-facing application

In this lab, you'll transform your multi-agent system from stateless interactions into an intelligent, memory-enabled system that learns and personalizes across all agents.

### Multi-Agent Memory Architecture for Lab 2
<div style="text-align:left">
    <img src="images/architecture_lab2_memory.png" width="75%"/>
</div>

*Multi-agent system with shared memory capabilities enabling cross-agent context and personalization.*

### Memory Integration Strategy

#### 🧠 **Shared Memory Architecture**
- **Customer Memory**: Shared across all agents for consistent personalization
- **Agent-Specific Memory**: Specialized memory for each agent type
- **Cross-Agent Context**: Memory context flows between agent handoffs

#### 🎯 **Memory Namespaces by Agent**
- **Orchestrator**: `support/orchestrator/{actorId}/routing` - Routing patterns and decisions
- **Customer Support**: `support/customer/{actorId}/preferences` - Customer preferences and behavior
- **Knowledge Base**: `support/technical/{actorId}/solutions` - Technical solutions and history
- **Shared Semantic**: `support/customer/{actorId}/semantic` - Cross-agent factual information

### Prerequisites

* **AWS Account** with appropriate permissions
* **Python 3.10+** installed locally
* **AWS CLI configured** with credentials
* **Amazon Nova Pro** enabled on [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
* **Lab 1 completed** - Multi-agent foundation system
* **AgentCore Memory** permissions configured

### Step 1: Install Dependencies and Import Libraries
Let's set up the memory-enhanced multi-agent system with automated dependency management.

In [ ]:
# Install dependencies for memory-enhanced multi-agent system
print("🧠 Installing dependencies for Memory-Enhanced Multi-Agent System...")
%pip install -r requirements.txt --upgrade-strategy only-if-needed -q
print("✅ Dependencies installed successfully!")
print("🚀 Ready to start Lab 2: Multi-Agent Memory Integration")
pip install --upgrade bedrock-agentcore bedrock-agentcore-starter-toolkit

In [ ]:
import logging
import uuid
import time
from typing import Dict, List, Any, Optional

# Import AgentCore Memory
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

# Import Strands for multi-agent system
from strands import Agent, tool
from strands.models import BedrockModel
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent

import boto3
from boto3.session import Session
from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter


# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# AWS Configuration
boto_session = Session()
REGION = boto_session.region_name
MODEL_ID = "us.amazon.nova-pro-v1:0"


# Define agent tools directly in the notebook
@tool(
    name="get_product_info",
    description="Get detailed technical specifications and information for electronics products"
)
def get_product_info(product_type: str) -> str:
    """
    Get detailed technical specifications and information for electronics products.

    Args:
        product_type: Electronics product type (e.g., 'laptops', 'smartphones', 'headphones', 'monitors')
    Returns:
        Formatted product information including warranty, features, and policies
    """
    # Mock product catalog - in real implementation, this would query a product database
    products = {
        "laptops": {
            "warranty": "1-year manufacturer warranty + optional extended coverage",
            "specs": "Intel/AMD processors, 8-32GB RAM, SSD storage, various display sizes",
            "features": "Backlit keyboards, USB-C/Thunderbolt, Wi-Fi 6, Bluetooth 5.0",
            "compatibility": "Windows 11, macOS, Linux support varies by model",
            "support": "Technical support and driver updates included"
        },
        "smartphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "5G/4G connectivity, 128GB-1TB storage, multiple camera systems",
            "features": "Wireless charging, water resistance, biometric security",
            "compatibility": "iOS/Android, carrier unlocked options available",
            "support": "Software updates and technical support included"
        },
        "headphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "Wired/wireless options, noise cancellation, 20Hz-20kHz frequency",
            "features": "Active noise cancellation, touch controls, voice assistant",
            "compatibility": "Bluetooth 5.0+, 3.5mm jack, USB-C charging",
            "support": "Firmware updates via companion app"
        },
        "monitors": {
            "warranty": "3-year manufacturer warranty",
            "specs": "4K/1440p/1080p resolutions, IPS/OLED panels, various sizes",
            "features": "HDR support, high refresh rates, adjustable stands",
            "compatibility": "HDMI, DisplayPort, USB-C inputs",
            "support": "Color calibration and technical support"
        }
    }
    product = products.get(product_type.lower())
    if not product:
        return f"Technical specifications for {product_type} not available. Please contact our technical support team for detailed product information and compatibility requirements."

    return f"Technical Information - {product_type.title()}:\n\n" \
           f"• Warranty: {product['warranty']}\n" \
           f"• Specifications: {product['specs']}\n" \
           f"• Key Features: {product['features']}\n" \
           f"• Compatibility: {product['compatibility']}\n" \
           f"• Support: {product['support']}"



@tool(
    name="get_return_policy",
    description="Get return policy information for a specific product category"
)
def get_return_policy(product_category: str) -> str:
    """
    Get return policy information for a specific product category.

    Args:
        product_category: Electronics category (e.g., 'smartphones', 'laptops', 'accessories')

    Returns:
        Formatted return policy details including timeframes and conditions
    """
    # Mock return policy database - in real implementation, this would query policy database
    return_policies = {
        "smartphones": {
            "window": "30 days",
            "condition": "Original packaging, no physical damage, factory reset required",
            "process": "Online RMA portal or technical support",
            "refund_time": "5-7 business days after inspection",
            "shipping": "Free return shipping, prepaid label provided",
            "warranty": "1-year manufacturer warranty included"
        },
         "laptops": {
            "window": "30 days", 
            "condition": "Original packaging, all accessories, no software modifications",
            "process": "Technical support verification required before return",
            "refund_time": "7-10 business days after inspection",
            "shipping": "Free return shipping with original packaging",
            "warranty": "1-year manufacturer warranty, extended options available"
        },
        "accessories": {
            "window": "30 days",
            "condition": "Unopened packaging preferred, all components included",
            "process": "Online return portal",
            "refund_time": "3-5 business days after receipt",
            "shipping": "Customer pays return shipping under $50",
            "warranty": "90-day manufacturer warranty"
        }
    }


@tool(
    name="web_search",
    description="Search the web for updated information"
)
def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.
    
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"
    

@tool(
    name="get_technical_support",
    description="Get technical support information from knowledge base"
)
def get_technical_support(issue: str) -> str:
    """Get technical support solutions"""
    solutions = {
        "overheating": "Check ventilation, clean fans, monitor CPU usage, consider thermal paste replacement",
        "battery": "Calibrate battery, check power settings, replace if over 2 years old",
        "performance": "Update drivers, check for malware, increase RAM if needed"
    }
    
    for key, solution in solutions.items():
        if key in issue.lower():
            return f"Technical Solution: {solution}"
    
    return "Please provide more details about the technical issue for specific troubleshooting steps."

# Import progress tracker for memory ID persistence
try:
    from lab_helpers.shared.utils import progress_tracker
    print("📊 Progress tracking available")
except ImportError:
    # Create a simple fallback progress tracker
    class SimpleProgressTracker:
        def __init__(self):
            self.progress = {'setup_status': {}}
        def save_progress(self): pass
    progress_tracker = SimpleProgressTracker()
    print("📊 Using fallback progress tracking")

print("📚 Multi-agent components loaded successfully!")

### Step 2: Create Multi-Agent Memory Infrastructure

We'll create a comprehensive memory system that supports our multi-agent architecture with different memory strategies for different agent types.

#### Memory Strategies for Multi-Agent System:

1. **Customer Preferences (USER_PREFERENCE)**: Learns customer behavior patterns across all agents
2. **Semantic Memory (SEMANTIC)**: Stores factual information accessible by all agents
3. **Routing Intelligence**: Orchestrator learns optimal routing patterns
4. **Technical Solutions**: Knowledge base agent remembers successful solutions

#### Multi-Tenant Memory Namespaces:
- `support/customer/{actorId}/preferences`: Customer preferences shared across agents
- `support/customer/{actorId}/semantic`: Factual information for all agents
- `support/orchestrator/{actorId}/routing`: Orchestrator routing decisions
- `support/technical/{actorId}/solutions`: Technical support solutions

In [ ]:
# Memory client setup
memory_client = MemoryClient(region_name=REGION)
memory_name = "CustomerSupportMemory"

def create_or_get_multi_agent_memory():
    """Create or retrieve multi-agent memory resource with comprehensive strategies"""
    
    try:
        memory_id = get_ssm_parameter("/app/reinvent/agentcore/memory_id")
        memory_client.gmcp_client.get_memory(memoryId=memory_id)
        print(f"✅ Using existing memory resource: {memory_id}")
        return memory_id
    except Exception:
        print("🔄 Creating new memory resource...")
    
    try:
        strategies = [
            {
                StrategyType.USER_PREFERENCE.value: {
                    "name": "CustomerPreferences",
                    "description": "Captures customer preferences and behavior",
                    "namespaces": ["support/customer/{actorId}/preferences"],
                }
            },
            {
                StrategyType.SEMANTIC.value: {
                    "name": "CustomerSupportSemantic",
                    "description": "Stores facts from conversations",
                    "namespaces": ["support/customer/{actorId}/semantic"],
                }
            },
        ]
        print("Creating AgentCore Memory resources. This will take 2-3 minutes...")
        print("While we wait, let's understand what's happening behind the scenes:")
        print("• Setting up managed vector databases for semantic search")
        print("• Configuring memory extraction pipelines")
        print("• Provisioning secure, multi-tenant storage")
        print("• Establishing namespace isolation for customer data")
        
        response = memory_client.create_memory_and_wait(
            name=memory_name,
            description="Customer support agent memory",
            strategies=strategies,
            event_expiry_days=90,
        )
        memory_id = response["id"]
        
        try:
            put_ssm_parameter("/app/reinvent/agentcore/memory_id", memory_id)
        except Exception as e:
            print(f"⚠️ Could not save memory ID: {e}")
        
        return memory_id
        
    except Exception as e:
        print(f"❌ Failed to create memory resource: {e}")
        return None


In [ ]:
memory_id = create_or_get_multi_agent_memory()
if memory_id:
    print("✅ AgentCore Memory created successfully!")
    print(f"Memory ID: {memory_id}")
else:
    print("Memory resource not created. Try Again !")


### Step 3: Implement Multi-Agent Memory Hooks

Now we'll create memory hooks that work across our multi-agent system. Each agent type will have specialized memory behavior while sharing common customer context.

#### Memory Hook Architecture:
- **Base Memory Hook**: Common memory operations for all agents
- **Orchestrator Memory Hook**: Routing intelligence and agent coordination
- **Customer Support Memory Hook**: Customer preferences and interaction history
- **Knowledge Base Memory Hook**: Technical solutions and documentation context

In [ ]:
class BaseMultiAgentMemoryHook(HookProvider):
    """Base memory hook for multi-agent system"""

    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str, agent_type: str):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.agent_type = agent_type
        
        # Get available namespaces
        self.namespaces = {
            strategy["type"]: strategy["namespaces"]
            for strategy in self.client.get_memory_strategies(self.memory_id)
        }

    def get_relevant_namespaces(self) -> List[str]:
        """Get namespaces relevant to this agent type"""
        relevant = []
        for strategy_type, namespaces in self.namespaces.items():
            for namespace in namespaces:
                # Always include customer namespaces
                if "customer" in namespace:
                    relevant.append(namespace.format(actorId=self.actor_id))
                # Include agent-specific namespaces
                elif self.agent_type.lower() in namespace:
                    relevant.append(namespace.format(actorId=self.actor_id))
        return relevant

    def retrieve_agent_context(self, event: MessageAddedEvent):
        """Retrieve context relevant to this agent"""
        messages = event.agent.messages
        if (
            messages[-1]["role"] == "user"
            and "toolResult" not in messages[-1]["content"][0]
        ):
            user_query = messages[-1]["content"][0]["text"]

            try:
                all_context = []
                relevant_namespaces = self.get_relevant_namespaces()

                for namespace in relevant_namespaces:
                    memories = self.client.retrieve_memories(
                        memory_id=self.memory_id,
                        namespace=namespace,
                        query=user_query,
                        top_k=2,
                    )
                    
                    for memory in memories:
                        if isinstance(memory, dict):
                            content = memory.get("content", {})
                            if isinstance(content, dict):
                                text = content.get("text", "").strip()
                                if text:
                                    namespace_type = namespace.split("/")[1]  # customer, orchestrator, technical
                                    all_context.append(f"[{namespace_type.upper()}] {text}")

                # Inject context into the query
                if all_context:
                    context_text = "\n".join(all_context)
                    original_text = messages[-1]["content"][0]["text"]
                    messages[-1]["content"][0]["text"] = (
                        f"Agent Context ({self.agent_type}):\n{context_text}\n\n{original_text}"
                    )
                    logger.info(f"[{self.agent_type}] Retrieved {len(all_context)} context items")

            except Exception as e:
                logger.error(f"[{self.agent_type}] Failed to retrieve context: {e}")

    def save_agent_interaction(self, event: AfterInvocationEvent):
        """Save interaction with agent-specific context"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last user query and agent response
                user_query = None
                agent_response = None

                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif (
                        msg["role"] == "user"
                        and not user_query
                        and "toolResult" not in msg["content"][0]
                    ):
                        user_query = msg["content"][0]["text"]
                        break

                if user_query and agent_response:
                    # Add agent type context to the interaction
                    enhanced_response = f"[{self.agent_type}] {agent_response}"
                    
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[
                            (user_query, "USER"),
                            (enhanced_response, "ASSISTANT"),
                        ],
                    )
                    logger.info(f"[{self.agent_type}] Saved interaction to memory")

        except Exception as e:
            logger.error(f"[{self.agent_type}] Failed to save interaction: {e}")

    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks for this agent"""
        registry.add_callback(MessageAddedEvent, self.retrieve_agent_context)
        registry.add_callback(AfterInvocationEvent, self.save_agent_interaction)
        logger.info(f"[{self.agent_type}] Memory hooks registered")

In [ ]:
class OrchestratorMemoryHook(BaseMultiAgentMemoryHook):
    """Specialized memory hook for orchestrator agent"""
    
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        super().__init__(memory_id, client, actor_id, session_id, "Orchestrator")
    
    def save_routing_decision(self, user_query: str, selected_agent: str, reasoning: str):
        """Save orchestrator routing decisions for learning"""
        try:
            routing_info = f"Query: {user_query}\nRouted to: {selected_agent}\nReasoning: {reasoning}"
            
            self.client.create_event(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=[
                    (f"Routing Decision: {routing_info}", "OTHER"),
                ],
            )
            logger.info(f"[Orchestrator] Saved routing decision: {selected_agent}")
        except Exception as e:
            logger.error(f"[Orchestrator] Failed to save routing decision: {e}")


class CustomerSupportMemoryHook(BaseMultiAgentMemoryHook):
    """Specialized memory hook for customer support agent"""
    
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        super().__init__(memory_id, client, actor_id, session_id, "CustomerSupport")


class KnowledgeBaseMemoryHook(BaseMultiAgentMemoryHook):
    """Specialized memory hook for knowledge base agent"""
    
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        super().__init__(memory_id, client, actor_id, session_id, "KnowledgeBase")
    
    def save_technical_solution(self, problem: str, solution: str, success: bool):
        """Save technical solutions for future reference"""
        try:
            solution_info = f"Problem: {problem}\nSolution: {solution}\nSuccess: {success}"
            
            self.client.create_event(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=[
                    (f"Technical Solution: {solution_info}", "OTHER"),
                ],
            )
            logger.info(f"[KnowledgeBase] Saved technical solution (success: {success})")
        except Exception as e:
            logger.error(f"[KnowledgeBase] Failed to save technical solution: {e}")

print("🔗 Multi-Agent Memory Hooks implemented successfully!")
print("  • Base memory operations for all agents")
print("  • Orchestrator routing intelligence")
print("  • Customer support preference learning")
print("  • Knowledge base solution persistence")

### Step 4: Seed Multi-Agent Memory with Historical Data

Let's seed our memory system with realistic multi-agent interactions to demonstrate how memory works across different agents and customer touchpoints.

In [ ]:
# Customer for this lab session
CUSTOMER_ID = "customer_multiagent_001"
SESSION_ID = str(uuid.uuid4())

def seed_multi_agent_memory():
    """Seed memory with multi-agent customer interactions"""
    
    # Historical multi-agent interactions
    interactions = [
        # Initial customer support interaction
        (
            "I need help with my MacBook Pro that's overheating during video editing. It's getting really hot and the fans are loud.",
            "[CustomerSupport] I can help with thermal management issues. Let me check your system specifications and provide some optimization tips. Your MacBook Pro model and usage pattern suggest this is likely related to intensive video processing workloads.",
        ),
        
        # Technical support handoff
        (
            "The basic tips didn't work. I'm still getting thermal throttling during 4K video exports in Final Cut Pro.",
            "[KnowledgeBase] For persistent thermal issues during 4K video exports, let me provide advanced troubleshooting steps including Activity Monitor analysis, thermal paste considerations, and professional video editing optimization settings.",
        ),
        
        # Customer preferences emerge
        (
            "I'm looking for a new laptop under $1500 for programming and light gaming. I prefer ThinkPad models and need good Linux compatibility.",
            "[CustomerSupport] Based on your preferences for ThinkPad models and Linux compatibility, I'd recommend the ThinkPad E series or T series within your budget. Both offer excellent development environments and gaming capabilities.",
        ),
        
        # Gaming headphone inquiry
        (
            "What's your return policy on gaming headphones? I need low latency for competitive FPS games like CS2 and Valorant.",
            "[CustomerSupport] Our gaming headphones have a 30-day return policy. For competitive FPS gaming, you'll want headphones with under 40ms latency. I can recommend several models that meet these requirements.",
        ),
        
        # Technical follow-up
        (
            "My MacBook is still having issues. Can you help me check if it's a hardware problem?",
            "[KnowledgeBase] Let's run comprehensive hardware diagnostics. Based on your previous thermal issues, we should check the cooling system, thermal sensors, and CPU performance under load. I'll guide you through Apple Diagnostics.",
        ),
    ]
    
    # Orchestrator routing decisions
    routing_decisions = [
        ("Thermal management and overheating issues", "KnowledgeBase", "Technical problem requiring specialized troubleshooting"),
        ("Product recommendations and pricing", "CustomerSupport", "General product inquiry within customer support scope"),
        ("Return policy questions", "CustomerSupport", "Policy-related inquiry handled by customer support"),
        ("Hardware diagnostics and advanced troubleshooting", "KnowledgeBase", "Complex technical issue requiring knowledge base expertise"),
    ]
    
    try:
        # Seed customer interactions
        print("📝 Seeding multi-agent customer interactions...")
        memory_client.create_event(
            memory_id=memory_id,
            actor_id=CUSTOMER_ID,
            session_id="historical_session_1",
            messages=[item for user_msg, assistant_msg in interactions for item in [(user_msg, "USER"), (assistant_msg, "ASSISTANT")]]
        )
        
        # Seed orchestrator routing intelligence (stored in semantic memory)
        print("🎯 Seeding orchestrator routing decisions...")
        for query, agent, reasoning in routing_decisions:
            routing_info = f"Routing Decision - Query: {query}\nRouted to: {agent}\nReasoning: {reasoning}"
            memory_client.create_event(
                memory_id=memory_id,
                actor_id=CUSTOMER_ID,
                session_id="routing_history",
                messages=[(routing_info, "OTHER")]
            )
        
        print("✅ Multi-agent memory seeded successfully!")
        print("📊 Seeded data includes:")
        print(f"  • {len(interactions)} customer interactions across agents")
        print(f"  • {len(routing_decisions)} orchestrator routing decisions")
        print("  • Cross-agent context and handoff patterns")
        print("\n⏳ Long-term memory processing will extract patterns automatically...")
        
    except Exception as e:
        print(f"⚠️ Error seeding multi-agent memory: {e}")

# Seed the memory
seed_multi_agent_memory()

### Step 5: Verify Multi-Agent Memory Processing

Let's check that our memory system has processed the seeded interactions and extracted meaningful patterns across our multi-agent system.

In [ ]:
def check_multi_agent_memory_processing():
    """Check if multi-agent memory processing is complete"""
    
    retries = 0
    max_retries = 6
    
    while retries < max_retries:
        try:
            preferences = memory_client.retrieve_memories(
                memory_id=memory_id,
                namespace=f"support/customer/{CUSTOMER_ID}/preferences",
                query="customer preferences and requirements"
            )
            
            semantic = memory_client.retrieve_memories(
                memory_id=memory_id,
                namespace=f"support/customer/{CUSTOMER_ID}/semantic",
                query="technical issues and solutions"
            )
            
            if preferences or semantic:
                print(f"✅ Memory processing complete!")
                
                if preferences:
                    print(f"\n🎯 Customer Preferences ({len(preferences)} items):")
                    for i, memory in enumerate(preferences[:3], 1):
                        if isinstance(memory, dict):
                            content = memory.get('content', {})
                            if isinstance(content, dict):
                                text = content.get('text', '')
                                print(f"  {i}. {text}")
                
                if semantic:
                    print(f"\n🧠 Semantic Memories ({len(semantic)} items):")
                    for i, memory in enumerate(semantic[:3], 1):
                        if isinstance(memory, dict):
                            content = memory.get('content', {})
                            if isinstance(content, dict):
                                text = content.get('text', '')
                                print(f"  {i}. {text}")
                
                return True
        
        except Exception:
            pass
        
        retries += 1
        if retries < max_retries:
            time.sleep(10)
    
    print("⚠️ Memory processing timeout. Continuing...")
    return False

memory_ready = check_multi_agent_memory_processing()


### Step 6: Create Memory-Enhanced Multi-Agent System

Now let's create our memory-enhanced multi-agent system. Each agent will have memory capabilities while maintaining their specialized roles from Lab 1.

In [ ]:
# Initialize memory hooks for each agent type (if memory is available)
if memory_id:
    orchestrator_memory = OrchestratorMemoryHook(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)
    customer_support_memory = CustomerSupportMemoryHook(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)
    knowledge_base_memory = KnowledgeBaseMemoryHook(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)
    print("✅ Memory hooks initialized")
else:
    orchestrator_memory = None
    customer_support_memory = None
    knowledge_base_memory = None
    print("⚠️ Running without memory hooks")

# Initialize the Bedrock model
model = BedrockModel(
    model_id=MODEL_ID,
    region_name=REGION
)

print("🤖 Creating memory-enhanced multi-agent system...")

# Customer Support Agent with optional memory
customer_support_agent = Agent(
    model=model,
    hooks=[customer_support_memory] if customer_support_memory else [],
    tools=[
        get_product_info,
        get_return_policy,
        web_search
    ],
    system_prompt="""
You are a Customer Support Agent with access to customer memory and preferences.
Use customer context to provide personalized recommendations and support.
Handle product inquiries, return policies, and general customer service.
Always acknowledge customer preferences and past interactions when relevant.
"""
)

# Knowledge Base Agent with optional memory
knowledge_base_agent = Agent(
    model=model,
    hooks=[knowledge_base_memory] if knowledge_base_memory else [],
    tools=[get_technical_support],
    system_prompt="""
You are a Technical Support Agent with access to customer technical history.
Use past technical interactions to provide contextual troubleshooting.
Handle complex technical issues, hardware problems, and advanced troubleshooting.
Reference previous solutions and build upon past technical interactions.
"""
)

# Enhanced Orchestrator Agent with memory and routing intelligence
class MemoryEnhancedOrchestrator:
    """Orchestrator with memory-based routing intelligence"""
    
    def __init__(self, memory_hook: OrchestratorMemoryHook):
        self.memory_hook = memory_hook
        self.customer_support = customer_support_agent
        self.knowledge_base = knowledge_base_agent
        
        # Orchestrator's own agent for routing decisions
        self.orchestrator_agent = Agent(
            model=model,
            hooks=[memory_hook] if memory_hook else [],
            system_prompt="""
You are an Orchestrator Agent with memory of past routing decisions.
Analyze customer queries and route them to the most appropriate agent:
- CustomerSupport: Product info, returns, policies, general inquiries
- KnowledgeBase: Technical issues, troubleshooting, hardware problems

Use your memory of past successful routing decisions to improve accuracy.
Consider customer history and preferences when making routing decisions.

Respond with: ROUTE_TO: [CustomerSupport|KnowledgeBase] - [reasoning]
"""
        )
    
    def route_query(self, query: str) -> tuple[Agent, str]:
        """Route query to appropriate agent using memory-enhanced decision making"""
        try:
            # Get routing decision from orchestrator
            response = self.orchestrator_agent(f"Route this query: {query}")
            # routing_response = response.get('content', [{}])[0].get('text', '')
            routing_response = str(response) if response else ''

            
            # Parse routing decision
            if "ROUTE_TO: CustomerSupport" in routing_response:
                selected_agent = self.customer_support
                agent_name = "CustomerSupport"
            elif "ROUTE_TO: KnowledgeBase" in routing_response:
                selected_agent = self.knowledge_base
                agent_name = "KnowledgeBase"
            else:
                # Default to customer support
                selected_agent = self.customer_support
                agent_name = "CustomerSupport"
                routing_response = "Default routing to CustomerSupport"
            
            # Save routing decision to memory (if available)
            if self.memory_hook:
                self.memory_hook.save_routing_decision(query, agent_name, routing_response)
            
            return selected_agent, agent_name
            
        except Exception as e:
            logger.error(f"Routing failed: {e}")
            return self.customer_support, "CustomerSupport"
    
    def handle_query(self, query: str) -> dict:
        """Handle customer query with memory-enhanced routing"""
        print(f"\n🎯 Orchestrator analyzing query with memory context...")
        
        # Route to appropriate agent
        selected_agent, agent_name = self.route_query(query)
        
        print(f"📍 Routed to: {agent_name}")
        
        # Execute query with selected agent
        response = selected_agent(query)
        
        return {
            'routed_to': agent_name,
            'response': response
        }

# Create the memory-enhanced orchestrator
orchestrator = MemoryEnhancedOrchestrator(orchestrator_memory)

print("✅ Memory-Enhanced Multi-Agent System Ready!")
print("\n🎯 System Capabilities:")
print("  • Memory-aware query routing")
print("  • Cross-agent customer context sharing")
print("  • Personalized agent responses")
print("  • Learning from past interactions")
print("  • Technical solution persistence")

### Step 7: Test Memory-Enhanced Multi-Agent System

Let's test our memory-enhanced system with queries that demonstrate how memory improves the multi-agent experience across different scenarios.

In [ ]:
def test_memory_enhanced_system():
    """Test the memory-enhanced multi-agent system"""
    
    test_queries = [
        {
            "query": "I'm looking for gaming headphones again. What do you recommend?",
            "expected_memory": "Should remember previous gaming headphone inquiry and low latency requirements"
        },
        {
            "query": "My MacBook is still overheating. Any new solutions?",
            "expected_memory": "Should remember previous thermal issues and troubleshooting attempts"
        },
        {
            "query": "Can you recommend a laptop for development work?",
            "expected_memory": "Should remember ThinkPad preference and Linux compatibility requirements"
        }
    ]
    
    print("🧪 Testing Memory-Enhanced Multi-Agent System")
    print("=" * 60)
    
    for i, test in enumerate(test_queries, 1):
        print(f"\n📝 Test {i}: {test['query']}")
        print(f"🎯 Expected Memory: {test['expected_memory']}")
        print("-" * 40)
        
        try:
            result = orchestrator.handle_query(test['query'])
            
            print(f"\n🤖 Agent Response ({result['routed_to']}):")
            # response_text = result['response'].get('content', [{}])[0].get('text', 'No response')
            response_text = str(result['response'])

            
            # Truncate long responses for readability
            if len(response_text) > 500:
                response_text = response_text[:500] + "..."
            
            print(response_text)
            
        except Exception as e:
            print(f"❌ Test failed: {e}")
        
        print("\n" + "=" * 60)

# Run the tests
test_memory_enhanced_system()

### Step 8: Explore Cross-Agent Memory Insights

Let's examine how our memory system has learned from the interactions and what insights it has gathered across our multi-agent system.

In [ ]:
def explore_cross_agent_memory():
    """Explore memory insights across the multi-agent system"""
    
    print("🔍 Cross-Agent Memory Analysis")
    print("=" * 50)
    
    # Analyze customer preferences
    try:
        preferences = memory_client.retrieve_memories(
            memory_id=memory_id,
            namespace=f"support/customer/{CUSTOMER_ID}/preferences",
            query="customer preferences and behavior patterns",
            top_k=5
        )
        
        if preferences:
            print(f"\n🎯 Customer Preferences Learned ({len(preferences)} insights):")
            print("-" * 40)
            for i, memory in enumerate(preferences, 1):
                if isinstance(memory, dict):
                    content = memory.get('content', {})
                    if isinstance(content, dict):
                        text = content.get('text', '')
                        print(f"  {i}. {text}")
    
    except Exception as e:
        print(f"⚠️ Could not retrieve preferences: {e}")
    
    # Analyze technical solutions
    try:
        technical = memory_client.retrieve_memories(
            memory_id=memory_id,
            namespace=f"support/customer/{CUSTOMER_ID}/semantic",
            query="technical issues and solutions",
            top_k=5
        )
        
        if technical:
            print(f"\n🔧 Technical Solutions Remembered ({len(technical)} solutions):")
            print("-" * 40)
            for i, memory in enumerate(technical, 1):
                if isinstance(memory, dict):
                    content = memory.get('content', {})
                    if isinstance(content, dict):
                        text = content.get('text', '')
                        print(f"  {i}. {text}")
    
    except Exception as e:
        print(f"⚠️ Could not retrieve technical solutions: {e}")
    
    # Check routing intelligence (if available)
    try:
        routing = memory_client.retrieve_memories(
            memory_id=memory_id,
            namespace=f"support/orchestrator/{CUSTOMER_ID}/routing",
            query="routing decisions and patterns",
            top_k=3
        )
        
        if routing:
            print(f"\n🎯 Orchestrator Routing Intelligence ({len(routing)} patterns):")
            print("-" * 40)
            for i, memory in enumerate(routing, 1):
                if isinstance(memory, dict):
                    content = memory.get('content', {})
                    if isinstance(content, dict):
                        text = content.get('text', '')
                        print(f"  {i}. {text}")
    
    except Exception as e:
        print(f"⚠️ Could not retrieve routing intelligence: {e}")
    
    print("\n✅ Cross-agent memory analysis complete!")

# Explore the memory insights
explore_cross_agent_memory()

## Lab 2 Summary: Multi-Agent Memory Integration

### 🎉 Congratulations! 

You've successfully enhanced your multi-agent customer support system with persistent memory capabilities. Here's what you've accomplished:

### ✅ Key Achievements

1. **Multi-Agent Memory Architecture**: Created a comprehensive memory system that works across multiple specialized agents

2. **Cross-Agent Context Sharing**: Enabled customer context and preferences to flow seamlessly between agents

3. **Intelligent Routing Memory**: Built an orchestrator that learns from past routing decisions to improve accuracy

4. **Specialized Memory Hooks**: Implemented agent-specific memory behaviors while maintaining shared customer context

5. **Persistent Learning**: Created a system that continuously learns and improves from customer interactions

### 🧠 Memory Capabilities Implemented

- **Customer Preferences**: System remembers customer preferences across all agent interactions
- **Technical Solutions**: Knowledge base agent builds upon previous troubleshooting attempts
- **Routing Intelligence**: Orchestrator learns optimal routing patterns for different query types
- **Cross-Session Continuity**: Customer context persists across different sessions and timeframes

### 🚀 What's Next?

In **Lab 3**, you'll add:
- **AgentCore Gateway**: Secure tool sharing across agents
- **Identity Management**: Multi-user support with proper isolation
- **Advanced Security**: Role-based access control for agent tools

### 💡 Key Takeaways

- Memory transforms stateless agents into intelligent, personalized assistants
- Multi-agent systems benefit significantly from shared memory architectures
- AgentCore Memory provides enterprise-grade persistence with minimal complexity
- Cross-agent memory sharing enables sophisticated customer experiences

### 🔧 Production Considerations

- Memory namespaces provide natural multi-tenancy for customer isolation
- Long-term memory processing happens asynchronously without blocking interactions
- Memory strategies can be customized for different business requirements
- Cross-agent context sharing scales to complex multi-agent workflows

**Ready for Lab 3?** Your memory-enhanced multi-agent system is now prepared for secure, scalable deployment with AgentCore Gateway!

In [ ]:
print("✅ Lab 2: Multi-Agent Memory Integration - COMPLETED!")
print("🎯 Ready for Lab 3: Multi-Agent Gateway & Identity")
print("\n📊 Lab 2 Statistics:")
print(f"  • Memory ID: {memory_id}")
print(f"  • Customer ID: {CUSTOMER_ID}")
print(f"  • Session ID: {SESSION_ID}")
print(f"  • Agents Enhanced: 3 (Orchestrator, CustomerSupport, KnowledgeBase)")
print(f"  • Memory Namespaces: 4 (Customer, Semantic, Routing, Technical)")